# Схема данных

| customer               |                                                                                                                                                       |
|------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------|
| customer_id            | Идентификатор клиента                                                                                                                                 |
| product_X              | Статус продукта. OPN - открыт, но не утилизирован. UTL - утилизирован. CLS - закрыт                                                                   |
| gender_cd              | Пол. M - мужской. F - женский                                                                                                                         |
| age                    | Возраст в годах                                                                                                                                       |
| marital_status_cd      | Семейный статус. См. словарь соответствия                                                                                                             |
| children_cnt           | Количество детей в штуках                                                                                                                             |
| first_session_dttm     | Дата и время первой сессии в приложении или личном кабинете на сайте                                                                                  |
| job_position_cd        | Категория занимаемой должности. См. словарь соответствия                                                                                              |
| job_title              | Занимаемая должность                                                                                                                                  |

| stories_reaction_train |                                                                                                                                                       |
|------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------|
| customer_id            | Идентификатор клиента                                                                                                                                 |
| story_id               | Идентификатор истории                                                                                                                                 |
| event_dttm             | Дата действия                                                                                                                                         |
| event                  | Тип действия. like - лайк или сохранение. view - глубокий просмотр (более 10 секунд). skip - пролистанная история (менее 5 секунд). dislike - дизлайк |

In [10]:
import org.apache.spark.rdd._

class SimpleCSVHeader(header:Array[String]) extends Serializable {
   
  val index = header.zipWithIndex.toMap
  def apply(array:Array[String], key:String) : String = {
      val curIndex = index(key)
      if (curIndex < array.size) {
          return array(curIndex)
      } else {
          return ""
      }
  }
}

val csvCustomer = sc.textFile("./customer_train.csv")  // original file
val dataCustomer = csvCustomer.map(line => line.split(",").map(elem => elem.trim)) //lines in rows
val headerCustomer = new SimpleCSVHeader(dataCustomer.first()) // we build our header with the first line
val customers = dataCustomer.filter(line => headerCustomer(line, "customer_id") != "customer_id") // filter the header out

val csvStories = sc.textFile("./stories_reaction_train.csv")  // original file
val dataStories = csvStories.map(line => line.split(",").map(elem => elem.trim)) //lines in rows
val headerStories = new SimpleCSVHeader(dataStories.first()) // we build our header with the first line
val stories = dataStories.filter(line => headerStories(line,"customer_id") != "customer_id") // filter the header out

import org.apache.spark.rdd._
defined class SimpleCSVHeader
csvCustomer: org.apache.spark.rdd.RDD[String] = ./customer_train.csv MapPartitionsRDD[1] at textFile at <console>:32
dataCustomer: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[2] at map at <console>:33
headerCustomer: SimpleCSVHeader = SimpleCSVHeader@7dab0178
customers: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[3] at filter at <console>:35
csvStories: org.apache.spark.rdd.RDD[String] = ./stories_reaction_train.csv MapPartitionsRDD[5] at textFile at <console>:37
dataStories: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[6] at map at <console>:38
headerStories: SimpleCSVHeader = SimpleCSVHeader@796b3b1
stories: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[7] at filter at ...


In [6]:
dataCustomer.take(10)

res3: Array[Array[String]] = Array(Array(customer_id, product_0, product_1, product_2, product_3, product_4, product_5, product_6, gender_cd, age, marital_status_cd, children_cnt, first_session_dttm, job_position_cd, job_title), Array(894436, "", "", "", "", "", UTL, "", M, 30.0, MAR, 0.0, 2018-03-20 09:10:16, 1, Неруководящий сотрудник - обсл. Персонал), Array(524526, "", UTL, "", "", "", UTL, "", F, 20.0, UNM, 0.0, 2017-03-29 20:38:45, 16), Array(498134, "", UTL, "", "", "", "", "", F, 25.0, UNM, 0.0, 2018-03-12 11:25:06, 22), Array(278941, "", "", UTL, CLS, "", UTL, UTL, M, 25.0, "", "", 2016-02-21 18:47:51, 16, Неруководящий сотрудник - специалист), Array(877312, "", UTL, "", "", "", "", "", F, 40.0, MAR, 0.0, 2018-03-07 11:17:02, 22), Array(821806, "", "", "", UTL, "", UTL, "", F, ...


1. Посчитать количество пользователей без детей

In [68]:
val cnt_child_free_person = customers
    .map(s => if (headerCustomer(s, "children_cnt") != "" && headerCustomer(s, "children_cnt").toDouble < 1) 1 else 0)
    .reduce((a,b) => (a + b)) 

cnt_child_free_person: Int = 37284


2. Посчитать долю пользователей старше 40 лет

In [73]:
var old_person_cnt = customers
    .map(s => if (headerCustomer(s, "age") != "" && headerCustomer(s, "age").toDouble > 40) 1 else 0)
    .aggregate((0,0))(
        (u,t) => (u._1 + t, u._2 + 1),
        (u1,u2) => (u1._1 + u2._1, u1._2 + u2._2)
    )

old_person_cnt._1.toDouble / old_person_cnt._2

old_person_cnt: (Int, Int) = (8425,50000)
res18: Double = 0.1685


3. Посчитать количество историй, которые лайкнули люди, утилизировавшие продукт 2

In [11]:
val utilization_2_prodRDD = customers
    .filter(s => (headerCustomer(s, "product_2") == "UTL"))
    .map(s => (headerCustomer(s, "customer_id"), 1))

stories
    .filter(s => (headerStories(s, "event") == "like"))
    .map(s => (headerStories(s, "customer_id"), headerStories(s, "story_id")))
    .join(utilization_2_prodRDD)
    .map(s => s._2._1)
    .distinct() // оставляем только уникальные итсории (а они могу повторяться, если у них будет разноее время например)
    .count

// customer_id story_id 1

utilization_2_prodRDD: org.apache.spark.rdd.RDD[(String, Int)] = MapPartitionsRDD[9] at map at <console>:35
res2: Long = 531


4. С помощью flatMap посчитать у каждого пользователя количество продуктов по статусу

In [45]:
customers
    .map(s => (headerCustomer(s, "customer_id"),
               s
               .aggregate((0, 0, 0))(
                   (u, t) => (u._1 + (if (t == "OPN") 1 else 0),
                              u._2 + (if (t == "UTL") 1 else 0),
                              u._3 + (if (t == "CLS") 1 else 0)),
                   (u1, u2) => (u1._1 + u2._2, u1._2 + u2._2, u1._3 + u2._3)
                   )
               )
        )//.count
    .collect()

res37: Array[(String, (Int, Int, Int))] = Array((894436,(0,1,0)), (524526,(0,2,0)), (498134,(0,1,0)), (278941,(0,3,1)), (877312,(0,1,0)), (821806,(0,2,0)), (782728,(0,1,0)), (540071,(0,1,0)), (66158,(0,2,0)), (804202,(0,2,0)), (323753,(0,2,0)), (244719,(0,2,0)), (110927,(1,2,0)), (307008,(0,1,2)), (643021,(0,1,0)), (739814,(0,2,0)), (816597,(0,1,0)), (387428,(1,1,0)), (134847,(0,2,0)), (650669,(0,2,0)), (210331,(0,2,0)), (143055,(0,2,0)), (106750,(0,2,0)), (392634,(0,2,0)), (195762,(0,1,1)), (902650,(0,1,0)), (761975,(0,1,0)), (505245,(0,1,0)), (480749,(0,1,0)), (44824,(0,1,0)), (11028,(0,2,0)), (239164,(0,1,0)), (830854,(0,1,1)), (303498,(0,2,0)), (478642,(0,4,1)), (442584,(0,1,0)), (341657,(0,2,0)), (663082,(0,2,0)), (623830,(0,1,0)), (802333,(0,1,0)), (598106,(0,0,0)), (762259,(0,1,0...


_в чате было написнао что flatMap можно не использовать )_

Далее во всех задачах для функций max\min будем использовать: 
```
rdd.max()(new Ordering[Tuple2[String, Int]]() {
  override def compare(x: (String, Int), y: (String, Int)): Int = 
      Ordering[Int].compare(x._2, y._2)
})
```

найденно тут: _https://stackoverflow.com/questions/26886275/how-to-find-max-value-in-pair-rdd_

5. Определить даты, в которые была наибольшая и наименьшая доля лайков историй от мужчин

In [16]:
val men_stories = customers
    .map(s => (headerCustomer(s, "customer_id"), if (headerCustomer(s, "gender_cd") == "M") 1 else 0))

val freq_mens_likes = stories
    .filter(s => headerStories(s, "event") == "like")
    .map(s => (headerStories(s, "customer_id"), headerStories(s, "event_dttm")))
    .join(men_stories)
    .map(s => (s._2._1.slice(0, 10), (s._2._2, 1))) //(data, 1 if like, 1)
    .reduceByKey((v1, v2) => (v1._1 + v2._1, v1._2 + v2._2)) // calc count 
    .map(s => (s._1, s._2._1.toDouble / s._2._2))

val ans_min = freq_mens_likes.max()(new Ordering[Tuple2[String, Double]]() {
  override def compare(x: (String, Double), y: (String, Double)): Int = 
      Ordering[Double].compare(x._2, y._2)
})._1

val ans_max = freq_mens_likes.min()(new Ordering[Tuple2[String, Double]]() {
  override def compare(x: (String, Double), y: (String, Double)): Int = 
      Ordering[Double].compare(x._2, y._2)
})._1                       

men_stories: org.apache.spark.rdd.RDD[(String, Int)] = MapPartitionsRDD[27] at map at <console>:34
freq_mens_likes: org.apache.spark.rdd.RDD[(String, Double)] = MapPartitionsRDD[35] at map at <console>:42
ans_min: String = 2018-06-29
ans_max: String = 2018-07-29


6. Среди тех, кто посмотрел историю 138, найдите id пользователя с максимальным количеством детей

In [121]:
val viewers138 = stories
    .filter(s => headerStories(s, "event") == "view" && headerStories(s, "story_id") == "138")
    .map(s => (headerStories(s, "customer_id"), 1))

customers
    .filter(s => headerCustomer(s, "children_cnt")!= "")
    .map(s => (headerCustomer(s, "customer_id"), headerCustomer(s, "children_cnt").toDouble))
    .join(viewers138) // (customer_id, (children_cnt, 1))
    .map(s => (s._1, s._2._1))
    .max()(new Ordering[Tuple2[String, Double]]() {
      override def compare(x: (String, Double), y: (String, Double)): Int = 
          Ordering[Double].compare(x._2, y._2)
    })

viewers138: org.apache.spark.rdd.RDD[(String, Int)] = MapPartitionsRDD[377] at map at <console>:36
res60: (String, Double) = (143885,3.0)


7. Найдите id истории с наибольшим отношением skip'ов к like'ам

In [13]:
stories
    .map(a => (headerStories(a, "story_id"), 
               if (headerStories(a, "event") == "like") (1.0, 0) else
               if (headerStories(a, "event") == "skip") (0.0, 1) 
               else (0.0, 0)
              )
        )
    .reduceByKey((v1, v2) => (v1._1 + v2._1, v1._2 + v2._2))
    .filter(s => s._2._1 > 0) // at least one like
    .map(s => (s._1, s._2._2 / s._2._1))
    .max()(new Ordering[Tuple2[String, Double]]() {
      override def compare(x: (String, Double), y: (String, Double)): Int = 
          Ordering[Double].compare(x._2, y._2)
    })._1

res4: String = 171


8. Напишите tail recursive функцию конкатенации листа листов

Доработаем код с семинара, по аналогии с вычислением факториала, за основу возьмем flatten

In [5]:
def flatten(list: List[Any]): List[Any]=
    list match{
        case (x : List[Any]) :: xs => flatten(x) ::: flatten(xs)
        case x :: xs => x :: flatten (xs)
        case Nil => Nil
    }

val nested = List(1, List(2,3), 4)
val flat = flatten(nested)

flatten: (list: List[Any])List[Any]
nested: List[Any] = List(1, List(2, 3), 4)
flat: List[Any] = List(1, 2, 3, 4)


In [7]:
import scala.annotation.tailrec

@tailrec
def flatten(arr: List[List[Any]], built: List[Any] = Nil): List[Any] = 
    arr match {
        case l :: (x: List[List[Any]]) => flatten(x, built ::: l)
        case l1 :: l2 => flatten(Nil, built ::: l1 ::: l2)
        case Nil => built
    }

val nested = List(List(2,3))
val flat = flatten(nested)

import scala.annotation.tailrec
flatten: (arr: List[List[Any]], built: List[Any])List[Any]
nested: List[List[Int]] = List(List(2, 3))
flat: List[Any] = List(2, 3)


P.S понятно что у нас упрощенная задача, и на таком примере: `((1,2),3, 4)` она не обязана работать